# Utah HB 15 - Example Household Impacts

This notebook shows how Utah's proposed Medicaid expansion repeal (HB 15) would affect different household types.

**Key takeaway:** Whether someone falls into the "coverage gap" or transitions to ACA depends primarily on their income relative to the Federal Poverty Level (FPL):
- Below 100% FPL → Coverage gap (no ACA available)
- 100-138% FPL → Can transition to ACA subsidies

**Note:** These examples assume federal Medicaid work requirements (80+ hours/month) are in effect, as modeled for 2027. Parents with children under 13 are exempt from work requirements.

In [ ]:
from policyengine_us import Simulation
from policyengine_core.periods import instant
from policyengine_core.reforms import Reform
import plotly.graph_objects as go
import numpy as np
from policyengine_core.charts import format_fig

YEAR = 2027

# PolicyEngine chart colors
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

def create_ut_medicaid_expansion_repeal():
    """Repeal Utah Medicaid expansion by setting income limit to -inf."""
    def modify_parameters(parameters):
        parameters.gov.hhs.medicaid.eligibility.categories.adult.income_limit.UT.update(
            start=instant(f"{YEAR}-01-01"),
            stop=instant("2100-12-31"),
            value=float("-inf"),
        )
        return parameters

    class reform(Reform):
        def apply(self):
            self.modify_parameters(modify_parameters)

    return reform

In [ ]:
def analyze_household(situation, name):
    """Analyze a household under baseline and reform."""
    baseline = Simulation(situation=situation)
    reformed = Simulation(situation=situation, reform=create_ut_medicaid_expansion_repeal())
    
    people = list(situation["people"].keys())
    
    print(f"{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    
    for person in people:
        age = situation["people"][person].get("age", {}).get(YEAR, "N/A")
        
        b_medicaid = baseline.calculate("medicaid", YEAR, map_to="person")[people.index(person)]
        b_ptc_elig = baseline.calculate("is_aca_ptc_eligible", YEAR)[people.index(person)]
        r_medicaid = reformed.calculate("medicaid", YEAR, map_to="person")[people.index(person)]
        r_ptc_elig = reformed.calculate("is_aca_ptc_eligible", YEAR)[people.index(person)]
        
        print(f"\n{person} (age {age}):")
        print(f"  Baseline:  Medicaid=${b_medicaid:,.0f}/yr, ACA eligible={b_ptc_elig}")
        print(f"  Reform:    Medicaid=${r_medicaid:,.0f}/yr, ACA eligible={r_ptc_elig}")
        
        if b_medicaid > 0 and r_medicaid == 0:
            if r_ptc_elig:
                print(f"  → LOSES MEDICAID, GAINS ACA ELIGIBILITY")
            else:
                print(f"  → FALLS INTO COVERAGE GAP")
    
    b_ptc = baseline.calculate("premium_tax_credit", YEAR)[0]
    r_ptc = reformed.calculate("premium_tax_credit", YEAR)[0]
    
    print(f"\nHousehold Premium Tax Credit:")
    print(f"  Baseline: ${b_ptc:,.0f}/yr")
    print(f"  Reform:   ${r_ptc:,.0f}/yr")
    print()

## Household 1: Single Adult Below Poverty Line → Coverage Gap

A single adult earning $12,000/year (about 75% FPL). Under current law, they qualify for Medicaid expansion. If expansion is repealed, they fall into the coverage gap because ACA subsidies only start at 100% FPL.

In [ ]:
household_1 = {
    "people": {
        "adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: 12_000},
            "monthly_hours_worked": {YEAR: 100},
        }
    },
    "tax_units": {"tax_unit": {"members": ["adult"]}},
    "spm_units": {"spm_unit": {"members": ["adult"]}},
    "households": {"household": {"members": ["adult"], "state_code": {YEAR: "UT"}}},
    "families": {"family": {"members": ["adult"]}},
    "marital_units": {"marital_unit": {"members": ["adult"]}},
}

analyze_household(household_1, "HOUSEHOLD 1: Single adult, $12k/yr (75% FPL) → COVERAGE GAP")

## Household 2: Single Adult Above Poverty Line → ACA Transition

A single adult earning $18,000/year (about 112% FPL). Under current law, they qualify for Medicaid expansion. If expansion is repealed, they can transition to ACA marketplace coverage with premium subsidies.

In [ ]:
household_2 = {
    "people": {
        "adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: 18_000},
            "monthly_hours_worked": {YEAR: 100},
        }
    },
    "tax_units": {"tax_unit": {"members": ["adult"]}},
    "spm_units": {"spm_unit": {"members": ["adult"]}},
    "households": {"household": {"members": ["adult"], "state_code": {YEAR: "UT"}}},
    "families": {"family": {"members": ["adult"]}},
    "marital_units": {"marital_unit": {"members": ["adult"]}},
}

analyze_household(household_2, "HOUSEHOLD 2: Single adult, $18k/yr (112% FPL) → ACA TRANSITION")

## Household 3: Parent with Child Below Poverty Line → Coverage Gap for Parent

A single parent with one child, earning $15,000/year (about 68% FPL for family of 2). The parent loses Medicaid expansion and falls into the coverage gap. The child remains eligible for Medicaid/CHIP (children's eligibility is separate from expansion).

In [ ]:
household_3 = {
    "people": {
        "parent": {
            "age": {YEAR: 30},
            "employment_income": {YEAR: 15_000},
            "monthly_hours_worked": {YEAR: 100},
        },
        "child": {"age": {YEAR: 8}},
    },
    "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
    "spm_units": {"spm_unit": {"members": ["parent", "child"]}},
    "households": {"household": {"members": ["parent", "child"], "state_code": {YEAR: "UT"}}},
    "families": {"family": {"members": ["parent", "child"]}},
    "marital_units": {"marital_unit": {"members": ["parent"]}},
}

analyze_household(household_3, "HOUSEHOLD 3: Single parent + child, $15k/yr (68% FPL) → PARENT IN COVERAGE GAP")

## Household 4: Couple with Children Above Poverty Line → ACA Transition

A married couple with two children, earning $38,000/year (about 115% FPL for family of 4). Both parents lose Medicaid expansion but can transition to ACA. Children remain on Medicaid/CHIP.

In [ ]:
household_4 = {
    "people": {
        "parent1": {
            "age": {YEAR: 40},
            "employment_income": {YEAR: 38_000},
            "monthly_hours_worked": {YEAR: 160},
        },
        "parent2": {
            "age": {YEAR: 38},
            "monthly_hours_worked": {YEAR: 0},
        },
        "child1": {"age": {YEAR: 12}},
        "child2": {"age": {YEAR: 7}},
    },
    "tax_units": {"tax_unit": {"members": ["parent1", "parent2", "child1", "child2"]}},
    "spm_units": {"spm_unit": {"members": ["parent1", "parent2", "child1", "child2"]}},
    "households": {"household": {"members": ["parent1", "parent2", "child1", "child2"], "state_code": {YEAR: "UT"}}},
    "families": {"family": {"members": ["parent1", "parent2", "child1", "child2"]}},
    "marital_units": {"marital_unit": {"members": ["parent1", "parent2"]}},
}

analyze_household(household_4, "HOUSEHOLD 4: Married couple + 2 kids, $38k/yr (115% FPL) → ACA TRANSITION")

## Visualizing the Impact

The graphs below compare how the reform affects two different household types across income levels:
- **Single adult** (no children)
- **Single parent with one child**

In [ ]:
# Generate data across income spectrum for both household types
incomes = np.arange(5000, 40001, 1000)
fpl_single = 16334  # FPL for 1 person in 2027
fpl_parent_child = 22138  # FPL for 2 people in 2027

# Single adult data
single_baseline_medicaid, single_baseline_ptc = [], []
single_reform_medicaid, single_reform_ptc = [], []

# Parent + child data
parent_baseline_medicaid, parent_baseline_ptc = [], []
parent_reform_medicaid, parent_reform_ptc = [], []

for income in incomes:
    # Single adult household
    single_situation = {
        "people": {"adult": {
            "age": {YEAR: 35},
            "employment_income": {YEAR: int(income)},
            "monthly_hours_worked": {YEAR: 100},
        }},
        "tax_units": {"tax_unit": {"members": ["adult"]}},
        "spm_units": {"spm_unit": {"members": ["adult"]}},
        "households": {"household": {"members": ["adult"], "state_code": {YEAR: "UT"}}},
        "families": {"family": {"members": ["adult"]}},
        "marital_units": {"marital_unit": {"members": ["adult"]}},
    }
    
    base = Simulation(situation=single_situation)
    ref = Simulation(situation=single_situation, reform=create_ut_medicaid_expansion_repeal())
    
    single_baseline_medicaid.append(base.calculate("medicaid", YEAR, map_to="person")[0])
    single_baseline_ptc.append(base.calculate("premium_tax_credit", YEAR)[0])
    single_reform_medicaid.append(ref.calculate("medicaid", YEAR, map_to="person")[0])
    single_reform_ptc.append(ref.calculate("premium_tax_credit", YEAR)[0])
    
    # Parent + child household
    parent_situation = {
        "people": {
            "parent": {
                "age": {YEAR: 30},
                "employment_income": {YEAR: int(income)},
                "monthly_hours_worked": {YEAR: 100},
            },
            "child": {"age": {YEAR: 8}},
        },
        "tax_units": {"tax_unit": {"members": ["parent", "child"]}},
        "spm_units": {"spm_unit": {"members": ["parent", "child"]}},
        "households": {"household": {"members": ["parent", "child"], "state_code": {YEAR: "UT"}}},
        "families": {"family": {"members": ["parent", "child"]}},
        "marital_units": {"marital_unit": {"members": ["parent"]}},
    }
    
    base = Simulation(situation=parent_situation)
    ref = Simulation(situation=parent_situation, reform=create_ut_medicaid_expansion_repeal())
    
    # Only count parent's medicaid (child stays on Medicaid/CHIP)
    parent_baseline_medicaid.append(base.calculate("medicaid", YEAR, map_to="person")[0])
    parent_baseline_ptc.append(base.calculate("premium_tax_credit", YEAR)[0])
    parent_reform_medicaid.append(ref.calculate("medicaid", YEAR, map_to="person")[0])
    parent_reform_ptc.append(ref.calculate("premium_tax_credit", YEAR)[0])

# Convert to arrays
single_baseline_medicaid = np.array(single_baseline_medicaid)
single_baseline_ptc = np.array(single_baseline_ptc)
single_reform_medicaid = np.array(single_reform_medicaid)
single_reform_ptc = np.array(single_reform_ptc)

parent_baseline_medicaid = np.array(parent_baseline_medicaid)
parent_baseline_ptc = np.array(parent_baseline_ptc)
parent_reform_medicaid = np.array(parent_reform_medicaid)
parent_reform_ptc = np.array(parent_reform_ptc)

# Calculate totals
single_baseline_total = single_baseline_medicaid + single_baseline_ptc
single_reform_total = single_reform_medicaid + single_reform_ptc
parent_baseline_total = parent_baseline_medicaid + parent_baseline_ptc
parent_reform_total = parent_reform_medicaid + parent_reform_ptc

In [ ]:
from plotly.subplots import make_subplots

# Create side-by-side charts
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Single Adult", "Single Parent + Child"),
    horizontal_spacing=0.1,
)

incomes_arr = np.array(incomes)

# Single adult chart (left)
fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=single_baseline_medicaid,
    mode='lines', name='Medicaid (Baseline)',
    line=dict(color=TEAL_ACCENT, width=2),
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=single_baseline_ptc,
    mode='lines', name='ACA PTC (Baseline)',
    line=dict(color=BLUE_PRIMARY, width=2),
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=single_reform_medicaid,
    mode='lines', name='Medicaid (Reform)',
    line=dict(color=TEAL_ACCENT, width=2, dash='dot'),
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=single_reform_ptc,
    mode='lines', name='ACA PTC (Reform)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot'),
), row=1, col=1)

# Parent + child chart (right) - hide duplicate legend entries
fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=parent_baseline_medicaid,
    mode='lines', name='Medicaid (Baseline)',
    line=dict(color=TEAL_ACCENT, width=2),
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=parent_baseline_ptc,
    mode='lines', name='ACA PTC (Baseline)',
    line=dict(color=BLUE_PRIMARY, width=2),
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=parent_reform_medicaid,
    mode='lines', name='Medicaid (Reform)',
    line=dict(color=TEAL_ACCENT, width=2, dash='dot'),
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=incomes_arr/1000, y=parent_reform_ptc,
    mode='lines', name='ACA PTC (Reform)',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot'),
    showlegend=False,
), row=1, col=2)

fig.update_layout(
    title="Utah HB 15: Health Benefits by Household Income",
    height=500,
    width=1000,
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
)

fig.update_xaxes(title_text="Household Income ($1,000s)")
fig.update_yaxes(title_text="Annual Benefit", tickformat="$,.0f", col=1)
fig.update_yaxes(tickformat="$,.0f", col=2)

fig = format_fig(fig)
fig.show()
fig.write_image("hb15_benefits_by_household.png", scale=2)

In [ ]:
# Change in benefits chart
single_change = single_reform_total - single_baseline_total
parent_change = parent_reform_total - parent_baseline_total

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Single Adult", "Single Parent + Child"),
    horizontal_spacing=0.1,
)

# Single adult
single_colors = [GRAY if c < 0 else BLUE_PRIMARY for c in single_change]
fig.add_trace(go.Bar(
    x=incomes_arr/1000, y=single_change,
    marker_color=single_colors,
    name="Single Adult",
    showlegend=False,
), row=1, col=1)

# Parent + child
parent_colors = [GRAY if c < 0 else BLUE_PRIMARY for c in parent_change]
fig.add_trace(go.Bar(
    x=incomes_arr/1000, y=parent_change,
    marker_color=parent_colors,
    name="Parent + Child",
    showlegend=False,
), row=1, col=2)

fig.add_hline(y=0, line=dict(color=DARK_GRAY, width=1))

fig.update_layout(
    title="Impact of Utah HB 15: Change in Health Benefits",
    height=500,
    width=1000,
    bargap=0.1,
)

fig.update_xaxes(title_text="Household Income ($1,000s)")
fig.update_yaxes(title_text="Change in Annual Benefits", tickformat="$,.0f", col=1)
fig.update_yaxes(tickformat="$,.0f", col=2)

fig = format_fig(fig)
fig.show()
fig.write_image("hb15_benefit_change.png", scale=2)

## Key Takeaways

1. **Coverage Gap**: At low incomes, adults lose Medicaid with no replacement. The coverage gap is wider for the parent+child household due to higher FPL thresholds.

2. **ACA Transition**: At higher incomes (above 100% FPL for their household size), people can transition to ACA subsidies which partially offset the Medicaid loss.

3. **Children Protected**: Children remain on Medicaid/CHIP regardless of the reform - only adult coverage is affected.